In [1]:
from HGCalImageAlgo3D import *
from utility_benchmark import *
from timeit import default_timer as timer
from tqdm import tqdm

%matplotlib inline
plt.style.use('classic')
plt.rc("figure",facecolor="w",figsize=(6,4))
plt.rc("font",size=10)
plt.rc("savefig",dpi=300)

Welcome to JupyROOT 6.10/00


In [2]:
DatasetDir  = '/Users/zihengchen/Documents/HGCal/workplace/data/'
DatasetFile = 'CMSSW93X_pt35_pu000'
df    = pd.read_hdf(DatasetDir+"input/"+DatasetFile+"_rechit.h5",key="table")
dfgen = pd.read_pickle(DatasetDir+"input/"+DatasetFile+"_gen.pkl")

# 1. Define Fitness

In [4]:
def TranslateDNA(DNA):
    para_upper = np.array([2,8,1])
    para_lower = np.array([0,2,0])
    para = DNA*(para_upper-para_lower)+para_lower
    return para

def DefineFitness(DNA):
    para = TranslateDNA(DNA)
    imagalgo = ImagingAlgo(AFFINITY_Z_ = para[0],
                           KERNAL_R_   = para[1],
                           KERNAL_EXPC_= para[2]
                          )
    
    dfresultclus = imagalgo.RunImagingAlgo(df,
                                           N=100,
                                           verb=False)
    
    gene,cluse,clusn = getoutputclust(DatasetDir,DatasetFile,
                                      N=100,deltarho=5,
                                      dfresultclus=dfresultclus,
                                      dfgen=dfgen)
    
    eff,ave = effsigma(cluse/gene)
    fitness = 1/eff
    return fitness

# 2. Define GA and Optimize

In [5]:
import sys
sys.path.append('../scripts/')
from GeneticAlgo import *

ga = GA(pop_size    = 40,
        DNA_size    = 3, 
        DNA_fitness = DefineFitness,
        cross_rate  = 0.8,
        mutate_rate = 0.01,
        noise_rate  = 0.4,
        noise_amp   = 0.02
       )

In [6]:
fitness_list,DNA_list,best_para_list = [],[],[]
for generation in tqdm(range(50)):
    ga.evolve()
    fitness  = ga.get_pop_fitness()
    DNA      = ga.pop
    best_DNA = ga.pop[np.argmax(fitness)]
    
    fitness_list.append(fitness)
    DNA_list.append(DNA)
    best_para_list.append(TranslateDNA(best_DNA))

100%|██████████| 50/50 [8:45:10<00:00, 607.75s/it]  


In [13]:
fitness_list   = np.array(fitness_list)
DNA_list       = np.array(DNA_list)
best_para_list = np.array(best_para_list)

np.savetxt('../data/parameter_GA/fitness.csv',fitness_list,delimiter=',')
np.savetxt('../data/parameter_GA/DNA.csv',DNA_list,delimiter=',')
#np.savetxt('../data/parameter_GA/best_para.csv',best_para_list,delimiter=',')

TypeError: Mismatch between array dtype ('float64') and format specifier ('%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e')

In [16]:
DNA_list.reshape(-1)

array([ 0.7355467 ,  0.85433776,  0.36212316, ...,  0.31083868,
        0.83141508,  0.44632404])